In [1]:
from food.tools import *
from food.paths import *
from food.psql import *
import requests
from food.qdrant import *
import torch
from torch.nn import CosineSimilarity
cos = CosineSimilarity(dim=1, eps=1e-08)
import numpy as np
import pandas as pd
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

import shapely
import shapely.geometry
from PIL import ImageFont, ImageDraw, Image

dev


In [7]:
import torchvision.transforms as T

In [8]:
#export
def crop_image_to_square(img):
    cropper = T.CenterCrop(min(img.size))
    return cropper(img)

In [9]:
series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])

In [10]:
foods = pd.read_sql("""select f.* , c.cluster 
                        from food.foods_prompted f
                        join food.foods_clusters c on (c.id = f.id)""",engine)

# foods = foods.drop(columns = ['clip'])
foods = foods.set_index('id')
foods = foods.dropna()

In [11]:
food_clips = series2tensor(foods['clip'])

In [13]:
def search_clip(url,head = 1):
    image_clip = requests.post(f'http://127.0.0.1:8182/image2vector/?url={url}').json()
    image_clip = torch.tensor(image_clip)
    df = foods.copy()
    df['score'] = cos(food_clips,image_clip)
    df = df.sort_values('score',ascending=False)[:head]

    return image_clip,df.reset_index()

In [12]:
url = 'https://image.shutterstock.com/z/stock-photo-healhty-vegan-lunch-bowl-avocado-quinoa-sweet-potato-tomato-spinach-and-chickpeas-vegetables-753381028.jpg'
image = get_image_from_url(url)
image = crop_image_to_square(image)
total_size = image.size[0]*image.size[1]

In [17]:
params = {'url':url}
masks = requests.post('http://127.0.0.1:8185/get_boxes/?',params = params).json()

In [18]:
masks

[{}, {}, {}, {}, {}]

In [ ]:

mask = np.array(mask)